In [1]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
from langchain_core.runnables.base import RunnableSequence
import openai,pandas as pd
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

api_key = os.getenv("Groq_API_Key")

llm = ChatGroq(model="llama3-8b-8192",api_key=api_key, temperature=0.3)

In [3]:
prompt_template = """
Given the following stock data for the past six months, predict the stock needs for the next month for each product:

{data}

The prediction should consider sales trends, product popularity, and stock shortages.

Additionally, provide recommendations to move products from stores with lower demand to stores with higher demand.

Provide predictions and recommendations in the following format:

- StoreID: 1, Product: Product A, Predicted Stock: <predicted_value>
- StoreID: 2, Product: Product B, Predicted Stock: <predicted_value>

Recommendations:
- Move <quantity> of Product A from Store <low_demand_store_id> to Store <high_demand_store_id>
"""

prompt = PromptTemplate(input_variables=["data"], template=prompt_template)


In [4]:
df = pd.read_csv(r"C:\Users\eDominer\Python Project\Sales Prediction\Agent\sales.csv")
df.head()


C:\Users\eDominer\AppData\Local\Temp\ipykernel_1248\4021793692.py:1: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\eDominer\Python Project\Sales Prediction\Agent\sales.csv")


,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
0,P0001,S0002,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
1,P0001,S0012,2017-01-02,1.0,5.30,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
2,P0001,S0013,2017-01-02,2.0,10.59,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
3,P0001,S0023,2017-01-02,0.0,0.00,6.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
4,P0001,S0025,2017-01-02,0.0,0.00,1.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN


In [5]:
df.drop(columns=['promo_type_1', 'promo_bin_1','promo_bin_2','promo_discount_2','promo_discount_type_2'], inplace=True)
df.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_2
0,P0001,S0002,2017-01-02,0.0,0.00,8.0,6.25,PR03
1,P0001,S0012,2017-01-02,1.0,5.30,0.0,6.25,PR03
2,P0001,S0013,2017-01-02,2.0,10.59,0.0,6.25,PR03
3,P0001,S0023,2017-01-02,0.0,0.00,6.0,6.25,PR03
4,P0001,S0025,2017-01-02,0.0,0.00,1.0,6.25,PR03


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19454838 entries, 0 to 19454837
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   product_id    object 
 1   store_id      object 
 2   date          object 
 3   sales         float64
 4   revenue       float64
 5   stock         float64
 6   price         float64
 7   promo_type_2  object 
dtypes: float64(4), object(4)
memory usage: 1.2+ GB


In [9]:
df.shape

(19454838, 8)

In [ ]:
df_string = df.to_string(index=False)

In [6]:
# import pandas as pd
# import matplotlib.pyplot as plt

# summary = df.groupby(['store_id', 'product_id']).sum()

# # Display the summary
# # print(summary)

# # Plotting the stock vs. sales for each product in each store
# fig, ax = plt.subplots()

# for store_id in df['store_id'].unique():
#     for product in df[df['store_id'] == store_id]['product_id'].unique():
#         data = df[(df['store_id'] == store_id) & (df['product_id'] == product)]
#         ax.plot(data['date'], data['stock'], label=f'Store {store_id} - Product {product} Stock')
#         ax.plot(data['date'], data['sales'], label=f'Store {store_id} - Product {product} Sales')

# ax.set_xlabel('date')
# ax.set_ylabel('Count')
# ax.set_title('Stock vs. Sales Comparison')
# ax.legend(loc='best')
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()


In [7]:
sequence = RunnableSequence(prompt, llm)

prediction = sequence.invoke({"data": df_string})

print(prediction.content)

C:\Users\eDominer\AppData\Local\Temp\ipykernel_5388\3670138312.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\eDominer\AppData\Local\Temp\ipykernel_5388\3670138312.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  prediction = llm_chain.run({"data": stock_data})


"Based on the provided data, I'll analyze the sales trends and product popularity to predict the stock needs for the next month. I'll also provide recommendations to move products from stores with lower demand to stores with higher demand.\n\n**Predictions:**\n\n1. StoreID: 1, Product: A, Predicted Stock: 65\n2. StoreID: 1, Product: B, Predicted Stock: 120\n3. StoreID: 2, Product: A, Predicted Stock: 80\n4. StoreID: 2, Product: B, Predicted Stock: 140\n5. StoreID: 3, Product: A, Predicted Stock: 75\n6. StoreID: 3, Product: B, Predicted Stock: 110\n\n**Recommendations:**\n\n1. Move 10 units of Product A from Store 3 to Store 1 (Store 1 has a higher demand for Product A, and Store 3 has a lower demand)\n2. Move 20 units of Product B from Store 2 to Store 1 (Store 1 has a higher demand for Product B, and Store 2 has a lower demand)\n\n**Analysis:**\n\nThe predictions are based on the following observations:\n\n* Product A has a consistent sales trend, with a slight decline over the past s

In [8]:
import markdown

html = markdown.markdown(prediction)


"<p>Based on the provided data, I'll analyze the sales trends and product popularity to predict the stock needs for the next month. I'll also provide recommendations to move products from stores with lower demand to stores with higher demand.</p>\n<p><strong>Predictions:</strong></p>\n<ol>\n<li>StoreID: 1, Product: A, Predicted Stock: 65</li>\n<li>StoreID: 1, Product: B, Predicted Stock: 120</li>\n<li>StoreID: 2, Product: A, Predicted Stock: 80</li>\n<li>StoreID: 2, Product: B, Predicted Stock: 140</li>\n<li>StoreID: 3, Product: A, Predicted Stock: 75</li>\n<li>StoreID: 3, Product: B, Predicted Stock: 110</li>\n</ol>\n<p><strong>Recommendations:</strong></p>\n<ol>\n<li>Move 10 units of Product A from Store 3 to Store 1 (Store 1 has a higher demand for Product A, and Store 3 has a lower demand)</li>\n<li>Move 20 units of Product B from Store 2 to Store 1 (Store 1 has a higher demand for Product B, and Store 2 has a lower demand)</li>\n</ol>\n<p><strong>Analysis:</strong></p>\n<p>The pre